#Introduction

In this repo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [116]:
%cd /content/drive/MyDrive/home/TheEasyYolo  #put the desired path in gdrive

/content/drive/MyDrive/home/TheEasyYolo


In [ ]:
!git clone https://github.com/MNaseerSubhani/TheEasyYolo-for-Industry-on-Colab.git

In [117]:
%cd TheEasyYolo-for-Industry-on-Colab

/content/drive/MyDrive/home/TheEasyYolo/TheEasyYolo-Plug-and-Play-for-Industry-on-Colab


In [136]:
import os
from shutil import copyfile


In [137]:
instance_name = "Yolo_training"

In [138]:
if not os.path.exists(instance_name):
  os.mkdir(instance_name)

In [139]:
yolo_ver = "YoloV4-tiny"      #The input should be YoloV4, YoloV4-tiny, YoloV3, YoloV3-tiny

In [140]:
if yolo_ver == "YoloV3-tiny":
  cfg_name = "yolov3-tiny.cfg"
elif yolo_ver == "YoloV3":
  cfg_name = "yolov3.cfg"
elif yolo_ver == "YoloV4-tiny":
  cfg_name = "yolov4-tiny.cfg"
elif yolo_ver == "YoloV4":
  cfg_name = "yolov4.cfg"


In [141]:
#set hyperparameters
num_of_classes = 1
channel = 1
sub_division = 16
width = 320
height = 256
batch = 64

In [142]:
cfg_file_name = f'{cfg_name[:-4]}_{instance_name}.cfg'
copyfile(os.path.join('./darknet/cfg/', cfg_name),os.path.join(instance_name, cfg_file_name))

'Yolo_training/yolov4-tiny_Yolo_training.cfg'

In [143]:
%cd {instance_name}
!sed -i 's/subdivisions=1/subdivisions=$sub_division/' {cfg_file_name}
!sed -i 's/width=416/width=$width/' {cfg_file_name}
!sed -i 's/batch=64/batch=$batch/' {cfg_file_name}
!sed -i 's/height=416/height=$height/' {cfg_file_name}
!sed -i 's/channel=3/channel=$channel/' {cfg_file_name}
!sed -i 's/classes=80/classes=$num_of_classes/' {cfg_file_name}
filters = int ((num_of_classes + 5) * 3)
!sed -i 's/filters=255/filters=$filters/' {cfg_file_name}
%cd ..

/content/drive/My Drive/home/TheEasyYolo/TheEasyYolo-Plug-and-Play-for-Industry-on-Colab/Yolo_training
/content/drive/My Drive/home/TheEasyYolo/TheEasyYolo-Plug-and-Play-for-Industry-on-Colab


In [95]:
copyfile('generate_test.py',os.path.join(instance_name, 'generate_test.py'))
copyfile('generate_train.py',os.path.join(instance_name, 'generate_train.py'))

'Yolo_training/generate_train.py'

In [96]:
%cd {instance_name}
!python generate_train.py
!python generate_test.py
%cd ..

/content/drive/MyDrive/home/TheEasyYolo/TheEasyYolo-Plug-and-Play-for-Industry-on-Colab/Yolo_training
/content/drive/MyDrive/home/TheEasyYolo/TheEasyYolo-Plug-and-Play-for-Industry-on-Colab


In [98]:
%cd {instance_name}
with open("train.names", "w") as f:   
    f.write("oil can")
    #f.write("\nother classe")
%cd ..

/content/drive/My Drive/home/TheEasyYolo/TheEasyYolo-Plug-and-Play-for-Industry-on-Colab/Yolo_training
/content/drive/My Drive/home/TheEasyYolo/TheEasyYolo-Plug-and-Play-for-Industry-on-Colab


In [100]:
%cd {instance_name}

if not os.path.exists('backup'):
  os.mkdir('backup')

with open("train.data", "w") as f:   
    f.write(f"classes={num_of_classes}")
    f.write(f"\ntrain = {os.path.join(os.getcwd(), 'data', 'train.txt')}")
    f.write(f"\nvalid = {os.path.join(os.getcwd(), 'data', 'test.txt')}")
    f.write(f"\nnames = {os.path.join(os.getcwd(), 'train.names')}")
    f.write(f"\nbackup = {os.path.join(os.getcwd(), 'backup/')}")
    #f.write("\nother classe")
%cd ..

/content/drive/My Drive/home/TheEasyYolo/TheEasyYolo-Plug-and-Play-for-Industry-on-Colab/Yolo_training
/content/drive/My Drive/home/TheEasyYolo/TheEasyYolo-Plug-and-Play-for-Industry-on-Colab


#Training

In [146]:
%cd darknet

/content/drive/My Drive/home/TheEasyYolo/TheEasyYolo-Plug-and-Play-for-Industry-on-Colab/darknet


In [102]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# Download pre-trained weights for the convolutional layers

In [105]:
if yolo_ver == "YoloV4" or yolo_ver == "YoloV4-tiny":
  !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
  pre_trained_weights = 'yolov4.conv.137'

In [104]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

In [106]:
!./darknet detector train ../{instance_name}/train.data ../{instance_name}/{cfg_file_name} {pre_trained_weights} -dont_show -map -clear

Streaming output truncated to the last 5000 lines.
 total_bbox = 73820, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 0.101373, iou_loss = 0.000000, total_loss = 0.101373 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.568496), count: 10, class_loss = 3.055079, iou_loss = 2.826329, total_loss = 5.881409 
 total_bbox = 73830, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.499432), count: 2, class_loss = 0.716537, iou_loss = 0.097477, total_loss = 0.814014 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.605215), count: 7, class_loss = 2.067132, iou_loss = 1.057922, total_loss = 3.125054 
 total_bbox = 73839, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.716263), count: 1, class_loss = 0.407953, io

Continue Training

In [108]:
!./darknet detector train ../{instance_name}/train.data ../{instance_name}/{cfg_file_name} ../{instance_name}/backup/{cfg_file_name[:-4] + '_last.weights'} -dont_show -map -clear

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    320 x 256 x   3 ->  160 x 128 x  32 0.035 BF
   1 conv     64       3 x 3/ 2    160 x 128 x  32 ->   80 x  64 x  64 0.189 BF
   2 conv     64       3 x 3/ 1     80 x  64 x  64 ->   80 x  64 x  64 0.377 BF
   3 route  2 		                       1/2 ->   80 x  64 x  32 
   4 conv     32       3 x 3/ 1     80 x  64 x  32 ->   80 x  64 x  32 0.094 BF
   5 conv     32       3 x 3/ 1     80 x  64 x  32 ->   80 x  64 x  32 0.094 BF
   6 route  5 4 	                           ->   80 x  64 x  64 
   7 conv     64       1 x 1/ 1     80 x  64 x  64

#Test

In [ ]:
%cd .. 

In [121]:
!ls Yolo_training

backup	generate_test.py   train.data	yolov4-tiny_Yolo_training.cfg
data	generate_train.py  train.names


In [119]:
cfg_file_name

'yolov4-tiny_Yolo_training.cfg'

In [144]:
%cd {instance_name}
!sed -i 's/batch=$batch/batch=1/' {cfg_file_name}
!sed -i 's/subdivisions=$sub_division/subdivisions=1/' {cfg_file_name}
%cd ..

/content/drive/My Drive/home/TheEasyYolo/TheEasyYolo-Plug-and-Play-for-Industry-on-Colab/Yolo_training
/content/drive/My Drive/home/TheEasyYolo/TheEasyYolo-Plug-and-Play-for-Industry-on-Colab


In [154]:
!./darknet detector test ../{instance_name}/train.data ../{instance_name}/{cfg_file_name} ../{instance_name}/backup/{cfg_file_name[:-4] + '_last.weights'} ../{instance_name}/data/train/frame135.jpg -thresh 0.1

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    320 x 256 x   3 ->  160 x 128 x  32 0.035 BF
   1 conv     64       3 x 3/ 2    160 x 128 x  32 ->   80 x  64 x  64 0.189 BF
   2 conv     64       3 x 3/ 1     80 x  64 x  64 ->   80 x  64 x  64 0.377 BF
   3 route  2 		                       1/2 ->   80 x  64 x  32 
   4 conv     32       3 x 3/ 1     80 x  64 x  32 ->   80 x  64 x  32 0.094 BF
   5 conv     32       3 x 3/ 1     80 x  64 x  32 ->   80 x  64 x  32 0.094 BF
   6 route  5 4 	                           ->   80 x  64 x  64 
   7 conv     64       1 x 1/ 1     80 x  64 x  64 ->   80 x  64 x  64 0.042 BF
   8 route  2 7 	     